In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
from mpi4py import MPI
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:3] 
n_proc = 8
rank = 0


In [4]:
%%px

folder = "/home/jhelsas/scratch"
filename = "ref-enstrophy-"+str(rank)+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
content = np.load(file)

w2 = np.zeros((nx,ny,nz), dtype='float32')
  
w2[:,:,:] = content['w2']
    
comm.Barrier(); t2=MPI.Wtime()

if(rank==0):
    print("Finished loading")
    sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))


[stdout:3] 
Finished loading
Load from disk: 1.05 seconds


In [6]:
%%px

folder = "/home/jhelsas/scratch"
filename = "ref-strainrate-"+str(rank)+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
content = np.load(file)

S2 = np.zeros((nx,ny,nz), dtype='float32')
    
S2[:,:,:] = content['S2']
    
comm.Barrier(); t2=MPI.Wtime()

if(rank==0):
    print("Finished loading")
    sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

[stdout:3] 
Finished loading
Load from disk: 1.06 seconds


In [7]:
%%px

w2 = 0.5*w2

In [14]:
%%px

avgO = np.average(w2)
avgOGl=np.zeros(1,dtype='float32')

comm.Allreduce([avgO,MPI.REAL],[avgOGl,MPI.REAL],op=MPI.SUM)
avgO = avgOGl[0]/nproc

########

avgE = np.average(S2)
avgEGl=np.zeros(1,dtype='float32')

comm.Allreduce([avgE,MPI.REAL],[avgEGl,MPI.REAL],op=MPI.SUM)
avgE = avgEGl[0]/nproc

########
    
avg = avgE

##########################

minw2 = w2.min()
maxw2 = w2.max()

minwGl=np.zeros(nproc,dtype='float32')
maxwGl=np.zeros(nproc,dtype='float32')

comm.Allgather([minw2,MPI.REAL],[minwGl,MPI.REAL])
comm.Allgather([maxw2,MPI.REAL],[maxwGl,MPI.REAL])

minO = minwGl.min()
maxO = maxwGl.max()

comm.Barrier()

##########################

minS2 = S2.min()
maxS2 = S2.max()

minS2Gl=np.zeros(nproc,dtype='float32')
maxS2Gl=np.zeros(nproc,dtype='float32')

comm.Allgather([minS2,MPI.REAL],[minS2Gl,MPI.REAL])
comm.Allgather([maxS2,MPI.REAL],[maxS2Gl,MPI.REAL])

minE = minS2Gl.min()
maxE = maxS2Gl.max()

comm.Barrier()

minJ = min(minO,minE)
maxJ = max(maxO,maxE)


if rank == 0:
    print(avgO,avgE,(avgE-avgO)/avgO)
    
if rank == 0:
    print("Separate : ",minO/avg,maxO/avg,minE/avg,maxE/avg)
    print("Joint : ",minJ/avg,maxJ/avg)

comm.Barrier()

if rank==0:
    print("log: ",np.log(minJ/avg),np.log(maxJ/avg))
    print("log_10: ",np.log(minJ/avg)/np.log(10),np.log(maxJ/avg)/np.log(10))

[stdout:3] 
(249.21774291992188, 249.21772766113281, -6.1226736442288224e-08)
('Separate : ', 1.4399783344273922e-07, 675.17004239278265, 3.9848981394448802e-05, 225.01035876849269)
('Joint : ', 1.4399783344273922e-07, 675.17004239278265)
('log: ', -15.753467583020129, 6.5149645738033355)
('log_10: ', -6.841644042147399, 2.829413164197959)


In [22]:
%%px

eta = 0.00280
x0 = y0 = z0 = 0.
dx = isotropic1024coarse['dx']
ner = int(1024*np.sqrt(3))
rbins = np.linspace(-0.5*dx,2*np.pi*np.sqrt(3)+0.5*dx,ner+1)

comm.Barrier(); t1=MPI.Wtime()

X = np.zeros((nx,ny,nz), dtype='float32')
Y = np.zeros((nx,ny,nz), dtype='float32')
Z = np.zeros((nx,ny,nz), dtype='float32')
r2 = np.zeros((nx,ny,nz), dtype='float32')

chi = np.zeros((nx,ny,nz), dtype='float32')

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocating vectors: {0:.2f} seconds\n'.format(t2-t1))

[stdout:3] Alocating vectors: 0.01 seconds


In [23]:
%%px

comm.Barrier(); t1=MPI.Wtime()
for i in range(nx):
    X[i,:,:] = (i+rank*nx)*isotropic1024coarse['dx']
    
for j in range(ny):
    Y[:,j,:] = j*isotropic1024coarse['dy']
    
for k in range(nz):
    Z[:,:,k] = k*isotropic1024coarse['dz']
    
r2[:,:,:] = X[:,:,:]**2+Y[:,:,:]**2+Z[:,:,:]**2

r2rt = np.sqrt(r2)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing the real domain for radial integration: {0:.2f} seconds\n'.format(t2-t1))

[stdout:3] Preparing the real domain for radial integration: 3.76 seconds


In [ ]:
%%px

dt = 0.05

thresholds = [1,2,3,4,5,6,7,10,15,20,30,50]

tboxes = []

for t in thresholds:
    comm.Barrier(); t1=MPI.Wtime()
    
    Xs = X[(w2 > t*avg)&(w2 < t*(1+dt)*avg)]
    Ys = Y[(w2 > t*avg)&(w2 < t*(1+dt)*avg)]
    Zs = Z[(w2 > t*avg)&(w2 < t*(1+dt)*avg)]
    
    print(Xs.shape)
    
    hist = np.zeros((Xs.shape[0],3))
    
    hist[:,0] = Xs[:]
    hist[:,1] = Ys[:]
    hist[:,2] = Zs[:]
    
    count = []
    scales = np.logspace(np.log(2*425*eta),np.log(0.1*42.5*eta), num=250, endpoint=True, base=np.e)
    #scales = [0.1*42.5*eta,0.3*42.5*eta]
    
    for L in scales:
        x1 = x0+isotropic1024coarse['lx']
        y1 = y0+isotropic1024coarse['ly']
        z1 = z0+isotropic1024coarse['lz']
        
        nx = int((x1-x0)/L)+1
        ny = int((y1-y0)/L)+1
        nz = int((z1-z0)/L)+1
        
        x1 = x0 + nx*L
        y1 = y0 + ny*L 
        z1 = z0 + nz*L
        
        H, edges = np.histogramdd(hist, bins=(nx,ny,nz), range=((x0,x1),(y0,y1),(z0,z1)))
        
        Hglobal = np.zeros(H.shape,dtype='float64')
        comm.Allreduce([H.astype('float64'),MPI.DOUBLE],[Hglobal,MPI.DOUBLE],op=MPI.SUM)
        
        Hn = Hglobal[:,:,:]
        Sn = Hglobal[:,:,:]
        Bn = Hglobal[:,:,:]
        ###############################################################
        # Here is where boxcouting and correlation dimension differs. #
        ###############################################################
        Bn[Bn>0] = 1
        numBox = np.sum(Bn)
        numPts = np.sum(Sn)
        Hn[:,:,:] = Hn[:,:,:]/(numPts)
        corrBox = np.sum(Hn**2)
        ###############################################################
        gbox = np.zeros(1,dtype='float32')
        gbox[0] = corrBox
        gbox = gbox[0]
        ###############################################################
        count.append(gbox)
    
    acount = np.array(count)
    
    tboxes.append(acount[:])
        
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Computing boxcounting numbers: {0:.2f} seconds\n'.format(t2-t1))

In [ ]:
%%px 

if rank==0:
    tfboxes = np.array(tboxes)
    np.savez('boxcount-shells-enstrophy-corr-dims.npz',tfboxes = tfboxes)

In [ ]:
%%px

dt = 0.05

thresholds = [1,2,3,4,5,6,7,10,15,20,30,50]

tboxes = []

for t in thresholds:
    comm.Barrier(); t1=MPI.Wtime()
    
    Xs = X[(S2 > t*avg)&(S2 < t*(1+dt)*avg)]
    Ys = Y[(S2 > t*avg)&(S2 < t*(1+dt)*avg)]
    Zs = Z[(S2 > t*avg)&(S2 < t*(1+dt)*avg)]
    
    print(Xs.shape)
    
    hist = np.zeros((Xs.shape[0],3))
    
    hist[:,0] = Xs[:]
    hist[:,1] = Ys[:]
    hist[:,2] = Zs[:]
    
    count = []
    scales = np.logspace(np.log(2*425*eta),np.log(0.1*42.5*eta), num=250, endpoint=True, base=np.e)
    #scales = [0.1*42.5*eta,0.3*42.5*eta]
    
    for L in scales:
        x1 = x0+isotropic1024coarse['lx']
        y1 = y0+isotropic1024coarse['ly']
        z1 = z0+isotropic1024coarse['lz']
        
        nx = int((x1-x0)/L)+1
        ny = int((y1-y0)/L)+1
        nz = int((z1-z0)/L)+1
        
        x1 = x0 + nx*L
        y1 = y0 + ny*L 
        z1 = z0 + nz*L
        
        H, edges = np.histogramdd(hist, bins=(nx,ny,nz), range=((x0,x1),(y0,y1),(z0,z1)))
        
        Hglobal = np.zeros(H.shape,dtype='float64')
        comm.Allreduce([H.astype('float64'),MPI.DOUBLE],[Hglobal,MPI.DOUBLE],op=MPI.SUM)
        
        Hn = Hglobal[:,:,:]
        Sn = Hglobal[:,:,:]
        Bn = Hglobal[:,:,:]
        ###############################################################
        # Here is where boxcouting and correlation dimension differs. #
        ###############################################################
        Bn[Bn>0] = 1
        numBox = np.sum(Bn)
        numPts = np.sum(Sn)
        Hn[:,:,:] = Hn[:,:,:]/(numPts)
        corrBox = np.sum(Hn**2)
        ###############################################################
        gbox = np.zeros(1,dtype='float32')
        gbox[0] = corrBox
        gbox = gbox[0]
        ###############################################################
        count.append(gbox)
    
    acount = np.array(count)
    
    tboxes.append(acount[:])
        
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Computing boxcounting numbers: {0:.2f} seconds\n'.format(t2-t1))

In [ ]:
%%px 

if rank==0:
    tfboxes = np.array(tboxes)
    np.savez('boxcount-shells-strainrate-corr-dims.npz',tfboxes = tfboxes)